In [207]:
import os
import csv


In [208]:
from syslog_processing import Processing


In [209]:
obj = Processing()


In [210]:
# input_csv_name = "GPN_Syslog_500.csv"
# input_csv_path = "GPN_Syslog_500.csv"
input_csv_path = 'processed_data/output_batch_1.csv'

STEP 1: Preprocess the Log Data for a particular router.

STEP 1a. Take router info from user

In [211]:
physical_site = input("Enter physical site id (eg-0001): ")
print(f"Physical site id chosen= {physical_site}")
geo_code = input("Enter geolocation code (eg-TLK): ").upper()
print(f"Geolocation code chosen= {geo_code}")
device_role = input("Enter device role (eg-CR for core router): ").upper()
print(f"Device role chosen= {device_role}")
device_function = input("Enter device function (eg-M14): ").upper()
print(f"Device model number chosen= {device_function}")
device_model_number = input("Enter device model number (eg-M14): ").upper()
print(f"Device model number chosen= {device_model_number}")
# device_importance = input("Enter device importance (eg-01 for primary router): ")
# print(f"Device importance chosen= {device_importance}")

Physical site id chosen= 0002
Geolocation code chosen= GST
Device role chosen= CR
Device model number chosen= M14


In [212]:
# desc = obj.get_specific_device_with_descriptions_formatted(input_csv_path= input_csv_path,physical_site_id='0001', geolocation_code='TLK', device_role='CR', device_model_number='M14', device_importance='01')
desc = obj.get_specific_device_with_descriptions_formatted(input_csv_path= input_csv_path,physical_site_id=physical_site, geolocation_code=geo_code, device_role=device_role, device_model_number=device_model_number)


In [213]:
desc

['gbn-0002-gst-cr--m14-02 -> failed to login through ssh 2 time',
 'gbn-0002-gst-cr--m14-02 -> log snmp/authenticationfailure is suppressed 1 in last 60 seconds.',
 'gbn-0002-gst-cr--m14-02 -> log snmp/snmp_authen_failed is suppressed 1 in last 60 seconds.',
 'gbn-0002-gst-cr--m14-02 -> the ssh user failed to login.',
 'gbn-0002-gst-cr--m14-02 -> the ssh user failed to login.',
 'gbn-0002-gst-cr--m14-02 -> failed to login through ssh 2 time',
 'gbn-0002-gst-cr--m14-02 -> failed to login through snmp.',
 'gbn-0002-gst-cr--m14-02 -> the source ip was locked because of the failure of login through snmp.',
 'gbn-0002-gst-cr--m14-02 -> the source ip was unlocked.',
 'gbn-0002-gst-cr--m14-02 -> failed to login through snmp.',
 'gbn-0002-gst-cr--m14-02 -> the source ip was locked because of the failure of login through snmp.',
 'gbn-0002-gst-cr--m14-02 -> the source ip was unlocked.',
 'gbn-0002-gst-cr--m14-02 -> the source ip was locked because of the failure of login through snmp.',
 'gbn-0

In [214]:
len(desc)

14

In [215]:
import pandas as pd

# Load log messages into a pandas DataFrame

df = pd.DataFrame(desc, columns=['log'])

In [216]:
df

,log
0,gbn-0002-gst-cr--m14-02 -> failed to login thr...
1,gbn-0002-gst-cr--m14-02 -> log snmp/authentica...
2,gbn-0002-gst-cr--m14-02 -> log snmp/snmp_authe...
3,gbn-0002-gst-cr--m14-02 -> the ssh user failed...
4,gbn-0002-gst-cr--m14-02 -> the ssh user failed...
5,gbn-0002-gst-cr--m14-02 -> failed to login thr...
6,gbn-0002-gst-cr--m14-02 -> failed to login thr...
7,gbn-0002-gst-cr--m14-02 -> the source ip was l...
8,gbn-0002-gst-cr--m14-02 -> the source ip was u...
9,gbn-0002-gst-cr--m14-02 -> failed to login thr...


STEP 2: Generate Embeddings with DistilBERT

Instead of classifying into rigid categories, we'll use DistilBERT to generate embeddings for each log message. These embeddings represent the semantics of each log message in a vector space.



In [217]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Tokenize the log messages
inputs = tokenizer(df['log'].tolist(), return_tensors='pt', padding=True, truncation=True)

# Generate embeddings using DistilBERT
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling to get sentence-level embeddings


c:\Users\prabhav.gupta\OneDrive - In2IT Technologies Pvt Ltd\Documents\AIOps\iSense\distilBert\venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Step 3: Clustering the Embeddings
Now that we have vector representations (embeddings) of each log message, we can apply clustering algorithms like K-Means, DBSCAN, or Agglomerative Clustering to group similar log messages together.

3.1 K-Means clustering (if you already know how many clusters)

In [218]:
# from sklearn.cluster import KMeans

# # Define the number of clusters (you can experiment with this number)
# num_clusters = 5

# # Apply K-Means clustering
# kmeans = KMeans(n_clusters=num_clusters, random_state=0)
# df['cluster'] = kmeans.fit_predict(embeddings.numpy())

3.2 DBSCAN (if you dont know how many clusters)

In [219]:
from sklearn.cluster import DBSCAN

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=2)
df['cluster'] = dbscan.fit_predict(embeddings.numpy())


In [220]:
df

,log,cluster
0,gbn-0002-gst-cr--m14-02 -> failed to login thr...,0
1,gbn-0002-gst-cr--m14-02 -> log snmp/authentica...,-1
2,gbn-0002-gst-cr--m14-02 -> log snmp/snmp_authe...,-1
3,gbn-0002-gst-cr--m14-02 -> the ssh user failed...,1
4,gbn-0002-gst-cr--m14-02 -> the ssh user failed...,1
5,gbn-0002-gst-cr--m14-02 -> failed to login thr...,0
6,gbn-0002-gst-cr--m14-02 -> failed to login thr...,2
7,gbn-0002-gst-cr--m14-02 -> the source ip was l...,3
8,gbn-0002-gst-cr--m14-02 -> the source ip was u...,4
9,gbn-0002-gst-cr--m14-02 -> failed to login thr...,2


Step 4: Visualizing and Analyzing the Clusters

Once you've clustered the log messages, you can inspect each cluster and identify potential patterns.

In [221]:
# Group by cluster and inspect log messages
final = []
anomaly = []
for cluster in df['cluster'].unique():
    print(f"Cluster {cluster}:")
    print(df[df['cluster'] == cluster]['log'].tolist())
    ans = df[df['cluster'] == cluster]['log'].tolist()
    if cluster == -1:
        anomaly.append(ans)
    else:    
        final.append(ans)
    print("\n")


Cluster 0:
['gbn-0002-gst-cr--m14-02 -> failed to login through ssh 2 time', 'gbn-0002-gst-cr--m14-02 -> failed to login through ssh 2 time']


Cluster -1:
['gbn-0002-gst-cr--m14-02 -> log snmp/authenticationfailure is suppressed 1 in last 60 seconds.', 'gbn-0002-gst-cr--m14-02 -> log snmp/snmp_authen_failed is suppressed 1 in last 60 seconds.']


Cluster 1:
['gbn-0002-gst-cr--m14-02 -> the ssh user failed to login.', 'gbn-0002-gst-cr--m14-02 -> the ssh user failed to login.']


Cluster 2:
['gbn-0002-gst-cr--m14-02 -> failed to login through snmp.', 'gbn-0002-gst-cr--m14-02 -> failed to login through snmp.']


Cluster 3:
['gbn-0002-gst-cr--m14-02 -> the source ip was locked because of the failure of login through snmp.', 'gbn-0002-gst-cr--m14-02 -> the source ip was locked because of the failure of login through snmp.', 'gbn-0002-gst-cr--m14-02 -> the source ip was locked because of the failure of login through snmp.']


Cluster 4:
['gbn-0002-gst-cr--m14-02 -> the source ip was unlocke

In [222]:
final

[['gbn-0002-gst-cr--m14-02 -> failed to login through ssh 2 time',
  'gbn-0002-gst-cr--m14-02 -> failed to login through ssh 2 time'],
 ['gbn-0002-gst-cr--m14-02 -> the ssh user failed to login.',
  'gbn-0002-gst-cr--m14-02 -> the ssh user failed to login.'],
 ['gbn-0002-gst-cr--m14-02 -> failed to login through snmp.',
  'gbn-0002-gst-cr--m14-02 -> failed to login through snmp.'],
 ['gbn-0002-gst-cr--m14-02 -> the source ip was locked because of the failure of login through snmp.',
  'gbn-0002-gst-cr--m14-02 -> the source ip was locked because of the failure of login through snmp.',
  'gbn-0002-gst-cr--m14-02 -> the source ip was locked because of the failure of login through snmp.'],
 ['gbn-0002-gst-cr--m14-02 -> the source ip was unlocked.',
  'gbn-0002-gst-cr--m14-02 -> the source ip was unlocked.',
  'gbn-0002-gst-cr--m14-02 -> the source ip was unlocked.']]

In [223]:
anomaly

[['gbn-0002-gst-cr--m14-02 -> log snmp/authenticationfailure is suppressed 1 in last 60 seconds.',
  'gbn-0002-gst-cr--m14-02 -> log snmp/snmp_authen_failed is suppressed 1 in last 60 seconds.']]

The clusters above tell us the seven major types of messages we are receiving in these logs. it will be helpful to deduplicate events and the unmber of clusters give an approximate idea of the distinct type of log messages.

In [224]:
types = len(final) + len(anomaly)

headers = ["DEVICE", "DE-DUPLICATED EVENTS", "FREQUENCY"]
cluster_rows = [elem[0] for elem in final]
anomaly_rows = [anomaly[0][i] for i in range(len(anomaly[0]))]


In [225]:
def get_frequencies(lst):
    frequency_dict = {}
    for item in lst:
        if item in frequency_dict:
            frequency_dict[item] += 1
        else:
            frequency_dict[item] = 1
    return frequency_dict

In [226]:
frequency_dict = get_frequencies(desc)

In [227]:
cluster_vertical = []
for c in cluster_rows:
    ans = []
    device, event = c.split('->')
    ans.append(device.strip())
    ans.append(event.strip())
    ans.append(frequency_dict.get(c,1))
    cluster_vertical.append(ans)


In [228]:
cluster_vertical

[['gbn-0002-gst-cr--m14-02', 'failed to login through ssh 2 time', 2],
 ['gbn-0002-gst-cr--m14-02', 'the ssh user failed to login.', 2],
 ['gbn-0002-gst-cr--m14-02', 'failed to login through snmp.', 2],
 ['gbn-0002-gst-cr--m14-02',
  'the source ip was locked because of the failure of login through snmp.',
  3],
 ['gbn-0002-gst-cr--m14-02', 'the source ip was unlocked.', 3]]

In [229]:
anomaly_vertical = []
for c in anomaly_rows:
    ans = []
    device, event = c.split('->')
    ans.append(device.strip())
    ans.append(event.strip())
    ans.append(frequency_dict.get(c,1))
    anomaly_vertical.append(ans)

In [230]:
anomaly_vertical

[['gbn-0002-gst-cr--m14-02',
  'log snmp/authenticationfailure is suppressed 1 in last 60 seconds.',
  1],
 ['gbn-0002-gst-cr--m14-02',
  'log snmp/snmp_authen_failed is suppressed 1 in last 60 seconds.',
  1]]

In [231]:

# data = [headers,cluster_rows + anomaly_rows]
data = [headers]
for i in cluster_vertical:
    data.append(i)
for j in anomaly_vertical:
    data.append(j)    

In [232]:
data

[['DEVICE', 'DE-DUPLICATED EVENTS', 'FREQUENCY'],
 ['gbn-0002-gst-cr--m14-02', 'failed to login through ssh 2 time', 2],
 ['gbn-0002-gst-cr--m14-02', 'the ssh user failed to login.', 2],
 ['gbn-0002-gst-cr--m14-02', 'failed to login through snmp.', 2],
 ['gbn-0002-gst-cr--m14-02',
  'the source ip was locked because of the failure of login through snmp.',
  3],
 ['gbn-0002-gst-cr--m14-02', 'the source ip was unlocked.', 3],
 ['gbn-0002-gst-cr--m14-02',
  'log snmp/authenticationfailure is suppressed 1 in last 60 seconds.',
  1],
 ['gbn-0002-gst-cr--m14-02',
  'log snmp/snmp_authen_failed is suppressed 1 in last 60 seconds.',
  1]]

In [233]:
# Specify the output file path
output_filepath = 'log_categorization.csv'

In [234]:
with open(output_filepath, mode='w', newline='', encoding='utf-8') as outfile:
    csv_writer = csv.writer(outfile)  # Initialize CSV writer
    csv_writer.writerows(data)

In [235]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill

In [236]:
# Create a new workbook and select the active worksheet
wb = Workbook()
ws = wb.active

In [237]:
# Define a fill color for the header and entry rows
header_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')  # Yellow
device_entry_fill = PatternFill(start_color='CCFFCC', end_color='CCFFCC', fill_type='solid')  # Light green
event_entry_fill = PatternFill(start_color='FFA07A', end_color='FFA07A', fill_type='solid')  # light red
frequence_entry_fill = PatternFill(start_color='ADD8E6', end_color='ADD8E6', fill_type='solid')  # Light blue


In [238]:

# Add data to the worksheet, color the rows, and adjust column widths
for row_index, row_data in enumerate(data):
    for col_index, cell_value in enumerate(row_data):
        cell = ws.cell(row=row_index + 1, column=col_index + 1, value=cell_value)
        
        # Color the header row
        if row_index == 0:  # First row is the header
            cell.fill = header_fill
        else:
            if col_index == 0:
                cell.fill = device_entry_fill
            elif col_index == 1:
                cell.fill = event_entry_fill
            elif col_index == 2:
                cell.fill = frequence_entry_fill



In [239]:
# Adjust column widths based on the length of the data
for col in ws.columns:
    max_length = 0
    column = col[0].column_letter  # Get the column letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = max_length + 2  # Add some padding
    ws.column_dimensions[column].width = adjusted_width

In [240]:
# Save the workbook to a file
output_filepath = 'log_categorization.xlsx'
wb.save(output_filepath)